In [1]:
%cd /ibex/user/slimhy/PADS/code/
# %env CUDA_LAUNCH_BLOCKING=1
"""
Extracting features into HDF5 files for each split.
"""
import torch

import numpy as np

from datasets.shapeloaders import CoMPaTManifoldDataset, ShapeNetDataset
from datasets.metadata import (
    get_compat_transform,
    get_shapenet_transform,
)
from datasets.metadata import SHAPENET_NAME_TO_SYNSET

/ibex/user/slimhy/PADS/code


In [2]:
def get_stuff(surface_points):
    # Get pc bounding box following each axis
    min_x = surface_points[:, :, 0].min()
    max_x = surface_points[:, :, 0].max()
    min_y = surface_points[:, :, 1].min()
    max_y = surface_points[:, :, 1].max()
    min_z = surface_points[:, :, 2].min()
    max_z = surface_points[:, :, 2].max()

    # Compute extents
    extents = [d.item() for d in [max_x - min_x, max_y - min_y, max_z - min_z]]
    return np.array(extents)

def flip_front_to_right(pc):
    """
    Rotate 90° around Y axis (from front-facing to right-facing).
    """
    full_transform = torch.tensor(
        [[0.0, 0.0, -1.0], [0.0, 1.0, 0.0], [1.0, 0.0, 0.0]],
        dtype=torch.float32,
        device=pc.device,
    )
    return torch.matmul(pc.squeeze(), full_transform).unsqueeze(0)

def get_datasets(active_class):
    # Initialize the latents
    shapenet_dataset = ShapeNetDataset(
        dataset_folder="/ibex/project/c2273/ShapeNet/",
        shape_cls=SHAPENET_NAME_TO_SYNSET[active_class],
        pc_size=2048,
    )

    # Initialize the latents
    compat_dataset = CoMPaTManifoldDataset(
        "/ibex/project/c2273/3DCoMPaT/manifold/",
        active_class,
        2048,
        normalize=False,
        sampling_method="surface",
        recenter_mesh=True,
        process_mesh=True
    )
    
    return shapenet_dataset, compat_dataset

# Iterate over both datasets and compute the average extents
def compute_avg_extents(dataset, transform, is_compat=False):
    extents = []
    for i in range(min(len(dataset), 100)):
        if is_compat:
            surface_points, _ = next(dataset[i])
        else:
            surface_points, _ = dataset[i]
        surface_points = transform(surface_points)
        extents.append(get_stuff(surface_points))
    return np.mean(extents, axis=0)

In [3]:
from datasets.metadata import COMPAT_MATCHED_CLASSES

all_extents = {}
for active_class in COMPAT_MATCHED_CLASSES:
    shapenet_dataset, compat_dataset = get_datasets(active_class)
    all_extents[active_class] = {
        "shapenet": compute_avg_extents(shapenet_dataset, transform=get_shapenet_transform(active_class)),
        "compat": compute_avg_extents(compat_dataset, transform=get_compat_transform(active_class), is_compat=True),
    }
    print(f"Done with {active_class}.")

Done with airplane.
Done with bag.
Done with basket.
Done with bed.
Done with bench.
Done with bird_house.
Done with boat.
Done with cabinet.
Done with car.
Done with chair.
Done with dishwasher.
Done with dresser.
Done with faucet.
Done with jug.
Done with lamp.
Done with love_seat.
Done with ottoman.
Done with planter.
Done with shelf.
Done with skateboard.
Done with sofa.
Done with sports_table.
Done with stool.
Done with table.
Done with trashcan.
Done with vase.


In [4]:
# Dump the extents to a file as a pickle
import util.misc as misc

misc.dump_pickle(all_extents, "/ibex/user/slimhy/PADS/data/extents.pkl")

In [8]:
all_extents['airplane']['shapenet']/all_extents['airplane']['compat']

array([0.97822517, 0.91619423, 0.99975245])

In [9]:
all_extents['airplane']['compat']

array([0.8908283 , 0.29077269, 0.93418145])

In [10]:
all_extents['airplane']['shapenet']

array([0.87143066, 0.26640427, 0.9339502 ])